In [1]:

# coding: utf-8

# In[10]:

import numpy as np
import theano
import theano.tensor as T
import lasagne
import librosa as lb

import matplotlib.pyplot as plt
get_ipython().magic(u'matplotlib inline')

import gzip
import pickle
import os
import os.path


# In[11]:

# Seed for reproduciblity
np.random.seed(42)


# In[12]:

# get_ipython().system(u'wget -N http://deeplearning.net/data/mnist/mnist.pkl.gz')




wav_dir = '/media/semionn/\xd0\xa5\xd0\xb0\xd1\x80\xd0\xb4/music_dataset_wav'
X = []
Y = []
clazzes = {}
for root, dirs, files in os.walk(wav_dir):
    X += list(map(lambda x: os.path.join(root, x), files))
    clazz = clazzes[root] if (root in clazzes) else len(clazzes)
    clazzes[root] = clazz
    for _ in files:
        Y.append(clazz-1)
    
from sklearn.cross_validation import train_test_split

X_train = X
y_train = Y
# X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.1)
y_val = y_train
X_val = X_train
# X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.11)
# print(set(y_train))

# print(X_train)
print(len(clazzes))

Couldn't import dot_parser, loading of dot files will not be possible.


Using gpu device 0: GeForce GTX 660 (CNMeM is disabled, cuDNN 4007)
/home/semionn/anaconda2/lib/python2.7/site-packages/theano/tensor/signal/downsample.py:6: UserWarning: downsample module has been moved to the theano.tensor.signal.pool module.
  "downsample module has been moved to the theano.tensor.signal.pool module.")


99


In [2]:
from random import randint

import functools

def memoize(obj):
    cache = obj.cache = {}

    @functools.wraps(obj)
    def memoizer(*args, **kwargs):
        key = str(args) + str(kwargs)
        if key not in cache:
            cache[key] = obj(*args, **kwargs)
        return cache[key]

    return memoizer


In [3]:
import librosa as lb
def f(x):
    return x.real

# f = np.vectorize(f)

@memoize
def get_file_spectre(filename):
    src, sr = lb.load (filename, sr=11025)
    return src, sr


In [4]:
def gen_sample(filename, n_batch=10):
    src, sr = get_file_spectre(filename)
    frame_size = sr*4
    res = []
    idx = np.random.choice(len(src) - frame_size, n_batch)
    for pos in idx:
        frame = src[pos: pos + frame_size]
        SRC = lb.stft (frame, n_fft=512, hop_length=256)
        res.append(f(SRC))
    return res

def batch_gen(X, y, n_batch):
    while True:
        idx = randint(0, len(X)-1)
        xs = []
        ys = []
        for x in gen_sample(X[idx], n_batch):
            xs.append(x)
            ys.append(y[idx])
        yield np.array(xs), np.array(ys).astype('int32')



In [37]:

l_in = lasagne.layers.InputLayer((None, 257, 173))

l_shape = lasagne.layers.ReshapeLayer(l_in, (-1, 1, 257, 173))
print(lasagne.layers.get_output_shape(l_shape))

l_conv = lasagne.layers.Conv2DLayer(l_shape, num_filters=16, stride=1, filter_size=(1, 173), pad='valid')
print(lasagne.layers.get_output_shape(l_conv))

l_pool = lasagne.layers.MaxPool2DLayer(l_conv, pool_size=(2, 1))
print(lasagne.layers.get_output_shape(l_pool))

l_conv2_31 = lasagne.layers.Conv2DLayer(l_pool, num_filters=16, stride=1, filter_size=(1, 1), pad='valid')
print(lasagne.layers.get_output_shape(l_conv2_31))

l_conv2_33 = lasagne.layers.Conv2DLayer(l_conv2_31, num_filters=16, stride=1, filter_size=(3, 1), pad=1)
print(lasagne.layers.get_output_shape(l_conv2_33))

l_conv2_36 = lasagne.layers.Conv2DLayer(l_conv2_33, num_filters=16, stride=1, filter_size=(1, 3), pad='valid')
print('l_conv2_36 %s' % str(lasagne.layers.get_output_shape(l_conv2_36)))

l_conv2_1 = lasagne.layers.Conv2DLayer(l_pool, num_filters=16, stride=1, filter_size=(1, 1), pad='valid')
print('l_conv2_1 %s' % str(lasagne.layers.get_output_shape(l_conv2_1)))

l_concat_1 = lasagne.layers.ConcatLayer([l_conv2_36, l_conv2_1], axis=1)
print('concat_1 %s' % str(lasagne.layers.get_output_shape(l_concat_1)))

l_pool2 = lasagne.layers.MaxPool2DLayer(l_concat_1, pool_size=(2, 1))
print(lasagne.layers.get_output_shape(l_pool2))

l_conv3 = lasagne.layers.Conv2DLayer(l_pool2, num_filters=16, stride=1, filter_size=(3, 1), pad='valid')
print(lasagne.layers.get_output_shape(l_conv3))

l_pool3 = lasagne.layers.MaxPool2DLayer(l_conv3, pool_size=(2, 1))
print(lasagne.layers.get_output_shape(l_pool3))

l_conv4 = lasagne.layers.Conv2DLayer(l_pool3, num_filters=16, stride=1, filter_size=(3, 1), pad='valid')
print(lasagne.layers.get_output_shape(l_conv4))

l_pool4 = lasagne.layers.MaxPool2DLayer(l_conv4, pool_size=(2, 1))
print(lasagne.layers.get_output_shape(l_pool4))

l_conv5 = lasagne.layers.Conv2DLayer(l_pool4, num_filters=16, stride=1, filter_size=(3, 1), pad='valid')
print(lasagne.layers.get_output_shape(l_conv5))

l_pool5 = lasagne.layers.Pool2DLayer(l_conv5, pool_size=(2, 1), mode='average_inc_pad')
print(lasagne.layers.get_output_shape(l_pool5))

l_out = lasagne.layers.DenseLayer(l_pool5,
                                  num_units=len(clazzes),
                                  nonlinearity=lasagne.nonlinearities.softmax)
layers = ['l_conv', 'l_pool', 'l_conv2_31', 'l_conv2_33', 'l_conv2_36', 'l_conv2_1', 'l_concat_1', 'l_pool2', 'l_conv3', 'l_pool3', 'l_conv4', 'l_pool4', 'l_conv5', 'l_pool5', 'l_out']
net = {}
net['l_conv'] = l_conv
net['l_pool'] = l_pool
net['l_conv2_31'] = l_conv2_31
net['l_conv2_33'] = l_conv2_33
net['l_conv2_36'] = l_conv2_36
net['l_conv2_1'] = l_conv2_1
net['l_concat_1'] = l_concat_1
net['l_pool2'] = l_pool2
net['l_conv3'] = l_conv3
net['l_pool3'] = l_pool3
net['l_conv4'] = l_conv4
net['l_pool4'] = l_pool4
net['l_conv5'] = l_conv5
net['l_pool5'] = l_pool5
net['l_out'] = l_out

layers = {k: net[k] for k in layers}

(None, 1, 257, 173)
(None, 16, 257, 1)
(None, 16, 128, 1)
(None, 16, 128, 1)
(None, 16, 128, 3)
l_conv2_36 (None, 16, 128, 1)
l_conv2_1 (None, 16, 128, 1)
concat_1 (None, 32, 128, 1)
(None, 32, 64, 1)
(None, 16, 62, 1)
(None, 16, 31, 1)
(None, 16, 29, 1)
(None, 16, 14, 1)
(None, 16, 12, 1)
(None, 16, 6, 1)


In [ ]:
# def get_samples(filename, offset=0):
#     src, sr = lb.load (filename, sr=11025)
#     frame_size = sr*4
#     res = []
#     for pos in range(len(src)//2, len(src), frame_size):
#         frame = src[pos: pos + frame_size]
#         SRC = lb.stft (frame, n_fft=512, hop_length=256)
#         res.append(f(SRC))
#         break
#     return res

In [38]:
# Compile and train the network.
# Accuracy is much better than the single layer network, despite the small number of filters.
X_sym = T.tensor3()
y_sym = T.ivector()

output = lasagne.layers.get_output(l_out, X_sym)
pred = output.argmax(-1)

loss = T.mean(lasagne.objectives.categorical_crossentropy(output, y_sym))

acc = T.mean(T.eq(pred, y_sym))

params = lasagne.layers.get_all_params(l_out)
grad = T.grad(loss, params)
updates = lasagne.updates.adam(grad, params, learning_rate=0.05)

f_train = theano.function([X_sym, y_sym], [loss, acc], updates=updates)
f_val = theano.function([X_sym, y_sym], [loss, acc])
f_predict = theano.function([X_sym], pred)
# values = pickle.load(open('weights0.200.pkl', 'rb'))
# lasagne.layers.set_all_param_values(layers.values(), values)


In [41]:

BATCH_SIZE = 20
N_BATCHES = 50
N_VAL_BATCHES = 10
EPOCH_CNT = 10

train_batches = batch_gen(X_train, y_train, BATCH_SIZE)
val_batches = batch_gen(X_val, y_val, BATCH_SIZE)

for epoch in range(EPOCH_CNT):
    train_loss = 0
    train_acc = 0
    for i in range(N_BATCHES):
#         print('Train batch #{}'.format(i))
        X1, y1 = next(train_batches)
#         print(X1.shape)
#         print(y1.shape)
        loss, acc = f_train(X1, y1)
        train_loss += loss
        train_acc += acc
    train_loss /= N_BATCHES
    train_acc /= N_BATCHES
    val_loss = 0
    val_acc = 0
    for i in range(N_VAL_BATCHES):
#         print('Validate batch #{}'.format(i))
        X1, y1 = next(val_batches)
        loss, acc = f_val(X1, y1)
        val_loss += loss
        val_acc += acc
    val_loss /= N_VAL_BATCHES
    val_acc /= N_VAL_BATCHES
    
    print('Epoch {}, Train (val) loss {:.03f} ({:.03f}) ratio {:.03f}'.format(
            epoch, train_loss, val_loss, val_loss/train_loss))
    print('Train (val) accuracy {:.03f} ({:.03f})'.format(train_acc, val_acc))

    all_weights = lasagne.layers.get_all_param_values(layers.values())
    pickle.dump(all_weights, open('weights{:.03f}.pkl'.format(val_acc), 'wb+'), pickle.HIGHEST_PROTOCOL)



Epoch 0, Train (val) loss 4.036 (4.856) ratio 1.203
Train (val) accuracy 0.060 (0.000)
Epoch 1, Train (val) loss 3.929 (3.660) ratio 0.932
Train (val) accuracy 0.080 (0.000)
Epoch 2, Train (val) loss 4.298 (4.233) ratio 0.985
Train (val) accuracy 0.000 (0.000)
Epoch 3, Train (val) loss 4.026 (4.348) ratio 1.080
Train (val) accuracy 0.040 (0.100)
Epoch 4, Train (val) loss 3.849 (4.312) ratio 1.120
Train (val) accuracy 0.060 (0.000)
Epoch 5, Train (val) loss 4.407 (3.887) ratio 0.882
Train (val) accuracy 0.000 (0.000)
Epoch 6, Train (val) loss 3.997 (4.435) ratio 1.109
Train (val) accuracy 0.000 (0.100)
Epoch 7, Train (val) loss 4.123 (4.530) ratio 1.099
Train (val) accuracy 0.020 (0.100)
Epoch 8, Train (val) loss 4.028 (4.653) ratio 1.155
Train (val) accuracy 0.040 (0.000)
Epoch 9, Train (val) loss 4.204 (4.563) ratio 1.086
Train (val) accuracy 0.020 (0.000)


In [ ]:
values = pickle.load(open('weights.pkl'))['param values']
lasagne.layers.set_all_param_values(layers, values)

In [ ]:
def gram_matrix(x):
    x = x.flatten(ndim=3)
    g = T.tensordot(x, x, axes=([2], [2]))
    return g


def content_loss(P, X, layer):
    p = P[layer]
    x = X[layer]
    
    loss = 1./2 * ((x - p)**2).sum()
    return loss


def style_loss(A, X, layer):
    a = A[layer]
    x = X[layer]
    
    A = gram_matrix(a)
    G = gram_matrix(x)
    
    N = a.shape[1]
    M = a.shape[2] * a.shape[3]
    
    loss = 1./(4 * N**2 * M**2) * ((G - A)**2).sum()
    return loss

def total_variation_loss(x):
    return (((x[:,:-1,:-1] - x[:,1:,:-1])**2 + (x[:,:-1,:-1] - x[:,:-1,1:])**2)**1.25).sum()

In [ ]:
style_samples = get_samples('converted_music/miniMusic/acrossu.mid')
target_samples = get_samples('converted_music/miniMusic/013705b_.mid')
style_music = style_samples[0]
target_music = target_samples[0]

In [ ]:
input_im_theano = T.matrix()
outputs = lasagne.layers.get_output(layers.values(), input_im_theano)

target_features = {k: theano.shared(output.eval({input_im_theano: target_music}))
                  for k, output in zip(layers.keys(), outputs)}
style_features = {k: theano.shared(output.eval({input_im_theano: style_music}))
                for k, output in zip(layers.keys(), outputs)}

In [ ]:
generated_music = theano.shared(floatX(np.random.uniform(-128, 128, (1, 257, 173))))

gen_features = lasagne.layers.get_output(layers.values(), generated_music)
gen_features = {k: v for k, v in zip(layers.keys(), gen_features)}

In [ ]:
# Define loss function
losses = []

# content loss
losses.append(0.001 * content_loss(target_features, gen_features, 'conv4_2'))

# style loss
losses.append(0.2e6 * style_loss(style_features, gen_features, 'conv1_1'))
losses.append(0.2e6 * style_loss(style_features, gen_features, 'conv2_1'))
losses.append(0.2e6 * style_loss(style_features, gen_features, 'conv3_1'))
losses.append(0.2e6 * style_loss(style_features, gen_features, 'conv4_1'))
losses.append(0.2e6 * style_loss(style_features, gen_features, 'conv5_1'))

# total variation penalty
losses.append(0.1e-7 * total_variation_loss(generated_music))

total_loss = sum(losses)